In [1]:
import time
from termcolor import colored
import torch

from modules.Dcls2d import Dcls2d
from modules.Dcls2dFull import Dcls2dFull


assert torch.cuda.is_available()
cuda_device = torch.device("cuda")  # device object representing GPU

in_channels = 1
out_channels = 1
kernel_size = (2,2)
dilation = (2,2)
stride = (1,1)
padding = (0,0)
groups = 1
bias = False

m = torch.nn.Conv2d(in_channels=in_channels,
              out_channels=out_channels,
              kernel_size=kernel_size,
              dilation=dilation,
              stride=stride,
              padding=padding,
              groups=groups,
              bias=bias).to(cuda_device)

n = Dcls2d(in_channels=in_channels,
              out_channels=out_channels,
              kernel_size=kernel_size,
              dilation=dilation,
              stride=stride,
              padding=padding,
              groups=groups,
              bias=bias).to(cuda_device)

o = Dcls2dFull(in_channels=in_channels,
              out_channels=out_channels,
              kernel_size=kernel_size,
              dilation=dilation,
              stride=stride,
              padding=padding,
              groups=groups,
              bias=bias).to(cuda_device)

X1 = torch.nn.Parameter(
                      torch.tensor([[[[1., 2., 3., 4.],
                                    [5., 6., 7., 8.], 
                                    [9., 10., 11., 12.],
                                    [13., 14., 15., 16.]]]],device=cuda_device),
                      requires_grad = True) 
X2 = torch.nn.Parameter(
                      torch.tensor([[[[1., 2., 3., 4.],
                                    [5., 6., 7., 8.], 
                                    [9., 10., 11., 12.],
                                    [13., 14., 15., 16.]]]],device=cuda_device),
                      requires_grad = True) 
X3 = torch.nn.Parameter(
                      torch.tensor([[[[1., 2., 3., 4.],
                                    [5., 6., 7., 8.], 
                                    [9., 10., 11., 12.],
                                    [13., 14., 15., 16.]]]],device=cuda_device),
                      requires_grad = True) 
m.weight = torch.nn.Parameter(
                      torch.tensor([[[[20., 40.],
                                    [60., 80.]]]],device=cuda_device),
                      requires_grad = True)
n.weight = torch.nn.Parameter(
                      torch.tensor([[[[20., 40.],
                                    [60., 80.]]]],device=cuda_device),
                      requires_grad = True)
o.weight = torch.nn.Parameter(
                      torch.tensor([[[[20., 40.],
                                    [60., 80.]]]],device=cuda_device),
                      requires_grad = True)

height_out = (4 + 2 * padding[0] - (dilation[0] * (kernel_size[0] - 1) + 1)) / stride[0] + 1;
print((dilation[0] * (kernel_size[0] - 1) + 1))
width_out = (4 + 2 * padding[1] - (dilation[1] * (kernel_size[1] - 1) + 1)) / stride[1] + 1;
back_truth = torch.nn.Parameter(
                      torch.tensor([[[[1., 2.],
                                    [4., 5.]]]],device=cuda_device),
                      requires_grad = True)

var1 = (m(X1) - back_truth).norm()
var2 = (n(X2) - back_truth).norm()
var3 = (o(X3) - back_truth).norm()


var1.backward();
var2.backward();
var3.backward();

3
tensor([[[[20.,  0., 40.],
          [ 0.,  0.,  0.],
          [60.,  0., 80.]]]], device='cuda:0',
       grad_fn=<SurrogateDilationFullBackward>)


In [2]:
print(X1.size())
print(m.weight.size())
print(n.weight.size())
print(o.weight.size())

print(m(X1).size())
print(m(X1))
print(n(X2).size())
print(n(X2))
print(o(X2).size())
print(o(X2))


print(m.weight.grad) 
print(n.weight.grad)
print(o.weight.grad)

torch.Size([1, 1, 4, 4])
torch.Size([1, 1, 2, 2])
torch.Size([1, 1, 2, 2])
torch.Size([1, 1, 2, 2])
torch.Size([1, 1, 2, 2])
tensor([[[[1560., 1760.],
          [2360., 2560.]]]], device='cuda:0',
       grad_fn=<CudnnConvolutionBackward>)
torch.Size([1, 1, 2, 2])
tensor([[[[1560.4507, 1760.4507],
          [2360.4507, 2560.4507]]]], device='cuda:0',
       grad_fn=<SurrogateDilationBackward>)
tensor([[[[20.,  0., 40.],
          [ 0.,  0.,  0.],
          [60.,  0., 80.]]]], device='cuda:0',
       grad_fn=<SurrogateDilationFullBackward>)
torch.Size([1, 1, 2, 2])
tensor([[[[20.,  0., 40.],
          [ 0.,  0.,  0.],
          [60.,  0., 80.]]]], device='cuda:0',
       grad_fn=<SurrogateDilationFullBackward>)
tensor([[[[1560.4646, 1760.4646],
          [2360.4646, 2560.4646]]]], device='cuda:0', grad_fn=<AddBackward0>)
tensor([[[[ 7.6718, 11.5944],
          [23.3621, 27.2847]]]], device='cuda:0')
tensor([[[[ 7.6717, 11.5943],
          [23.3621, 27.2847]]]], device='cuda:0')
tensor([

In [3]:
R1 = torch.ceil(n.P1) - n.P1
R2 = torch.ceil(n.P2) - n.P2

print((1 - R1) * (1-R2))
print((R1) * (1-R2))
print((1 - R1) * (R2))
print((R1) * (R2))

tensor([[[1., 1.],
         [1., 1.]]], device='cuda:0', grad_fn=<MulBackward0>)
tensor([[[0., 0.],
         [0., 0.]]], device='cuda:0', grad_fn=<MulBackward0>)
tensor([[[0., 0.],
         [0., 0.]]], device='cuda:0', grad_fn=<MulBackward0>)
tensor([[[0., 0.],
         [0., 0.]]], device='cuda:0', grad_fn=<MulBackward0>)


In [4]:
print(o.P1)

Parameter containing:
tensor([[[[-2., -2.],
          [ 0.,  0.]]]], device='cuda:0', requires_grad=True)
